In [ ]:
!apt-get -y install -qq tesseract-ocr tesseract-ocr-kor
!pip -q install pymupdf pillow pytesseract google-api-python-client google-auth-httplib2 google-auth-oauthlib opencv-python-headless

Selecting previously unselected package tesseract-ocr-kor.
(Reading database ... 126371 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-kor_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-kor (1:4.00~git30-7274cfa-1.1) ...
Setting up tesseract-ocr-kor (1:4.00~git30-7274cfa-1.1) ...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 61.4 MB/s eta 0:00:00


In [ ]:
from google.colab import auth
auth.authenticate_user()
print("Authenticated.")

Authenticated.


In [10]:
# ====== 반드시 바꿔 넣기 ======
SOURCE_FOLDER_ID = "1JccGWwnFlyPgkQA8ZBibZZhUap5hWhSF"   # DB 개설할때와 동일합니다.: pdf가 있는 곳의 파일 ID를 작성해주세요.TEST/HW 별로 교체해서 사용하시면 됩니다. # Replace "PDF" with your actual Google Drive folder ID
DEST_FOLDER_ID   = "1G4ygIz6Dw3ej1M_qj3XNu8Fvtdmn3IRu"   # DB 개설할때와 동일합니다.: 사진을 저장할 폴더 ID를 작성해주세요. 신규 개설을 해주시면 됩니다. 명칭은 상관 없습니다.
# =================================

# ---- 옵션(필요 시만 조정) ----
PROCESS_FIRST_PAGE_ONLY = True   # True=1페이지만 처리, False=모든 페이지
ZOOM = 2.6                       # 렌더 배율(2.2~3.0 권장; 작게 보이면 올리기)
ROW_KEYWORDS = ["comment", "코멘트"]  # 키워드(영/한)
OCR_LANGS = "eng+kor"            # OCR 언어_한국어 영어 무관하게 그냥 인풋
OCR_MIN_CONF = 50                # OCR 신뢰도 임계값(낮출수록 더 민감한데 더 낮추면 시간 너무 오래걸려....)
OCR_PSM = 6                      # 6=블록/단락, 7=한 줄 인식
MIN_HLINE_RATIO = 0.55           # 가로선 최소 길이(이미지 폭 대비)_PDF 파일 이미지 크기 계산해서 넣음.
LEFT_BAND_RATIO = 0.4            # 행의 '왼쪽'만 OCR (라벨이 오른쪽이면 0.6~0.8)
ROW_PAD_UP_PT = 2                # 행 위 여유(pt)_포인트로 2 여백 남기기
ROW_PAD_DOWN_PT = 2              # 행 아래 여유(pt)_포인트로 2 여백 남기기
# -----------------------------
# 이하 모든 구글 드라이브와 활용해야 하는 함수들 전부 import로 넣음.
import io, time, math, re
from typing import List, Dict, Optional, Tuple
import numpy as np
import cv2
import fitz  # PyMuPDF
from PIL import Image, ImageDraw
import pytesseract
from pytesseract import Output
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload, MediaIoBaseUpload

# ---- Drive API ---- -> Drive API 명령어 함수 구성 건드리지 말 것! 본인이 고칠 수 있다면 위 아래 꼭 살펴봐주세요.
drive = build("drive", "v3")

def list_children(folder_id: str, mime: Optional[str]=None) -> List[Dict]:
    q = f"'{folder_id}' in parents and trashed=false"
    if mime: q += f" and mimeType='{mime}'"
    out, tok = [], None
    while True:
        resp = drive.files().list(q=q, fields="nextPageToken,files(id,name,mimeType)", pageToken=tok).execute()
        out += resp.get("files", [])
        tok = resp.get("nextPageToken")
        if not tok: break
    return out

def list_pdfs_recursive(root_id: str) -> List[Dict]:
    stack, pdfs = [root_id], []
    while stack:
        cur = stack.pop()
        for f in list_children(cur, "application/vnd.google-apps.folder"):
            stack.append(f["id"])
        pdfs += list_children(cur, "application/pdf")
    return pdfs

def download_pdf_bytes(file_id: str) -> bytes:
    req = drive.files().get_media(fileId=file_id)
    buf = io.BytesIO()
    downloader = MediaIoBaseDownload(buf, req)
    done = False
    while not done:
        _, done = downloader.next_chunk()
    return buf.getvalue()

def upload_png_bytes(data: bytes, filename: str, folder_id: str):
    media = MediaIoBaseUpload(io.BytesIO(data), mimetype="image/png", resumable=False)
    drive.files().create(media_body=media, body={"name": filename, "parents": [folder_id]}).execute()


# ==== Comment 행(가로줄) 찾기: 텍스트 추출에 의존하지 않는 OpenCV+OCR 방식 ==== 지피티야 고마워....
def _norm_token(s: str) -> str:
    return re.sub(r"[^0-9a-zA-Z가-힣]", "", (s or "").lower())

def _opencv_horizontal_lines(gray: np.ndarray, min_len_ratio: float) -> list[int]:
    thr = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    binimg = 255 - thr if np.mean(thr) > 128 else thr  # 선 강조
    W = gray.shape[1]
    k = max(1, W // 60)  # 폭에 비례한 커널
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (k, 1))
    morph = cv2.morphologyEx(binimg, cv2.MORPH_OPEN, kernel, iterations=1)
    contours, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    ys = []
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        if w >= int(W * min_len_ratio) and h <= 4:
            ys.append(y)
    return sorted(set(ys))

def _ocr_has_keyword(pil_img: Image.Image) -> bool:
    data = pytesseract.image_to_data(pil_img, lang=OCR_LANGS, config=f"--psm {OCR_PSM}", output_type=Output.DICT)
    for i in range(len(data["text"])):
        txt = _norm_token(data["text"][i])
        if not txt:
            continue
        try: conf = float(data["conf"][i])
        except: conf = 0
        if conf < OCR_MIN_CONF:
            continue
        for kw in ROW_KEYWORDS:
            if _norm_token(kw) in txt:
                return True
    return False

def find_comment_row_band(page: fitz.Page) -> Optional[Tuple[float,float]]:
    # 1) 페이지 고해상도 렌더링하기. Comment 라는 셀 찾을 수 있게 최대한 땡겨 보는 것
    mat = fitz.Matrix(ZOOM, ZOOM)
    pix = page.get_pixmap(matrix=mat, alpha=False)
    Wpx, Hpx = pix.width, pix.height
    img = Image.frombytes("RGB", (Wpx, Hpx), pix.samples)
    gray = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)

    # 2) 긴 수평선(y 픽셀) 검출 → 행 경계 후보 : 수평선을 전부 지울거라 이렇게 넣었어요.
    y_lines = _opencv_horizontal_lines(gray, MIN_HLINE_RATIO)
    if len(y_lines) < 2:
        return None
    y_lines = sorted(y_lines)

    # 3) 각 '행 밴드'의 왼쪽 부분만 OCR해서 키워드 있는 행 찾기 : OCR이 성공해야 이게 적용이 되는데,,, 안되는 경우도 있는 거 같음. 규격 문제로 추정
    left_w = int(Wpx * LEFT_BAND_RATIO)
    for i in range(len(y_lines)-1):
        top_px, bot_px = y_lines[i], y_lines[i+1]
        if bot_px - top_px < 8:  # 너무 얇은 밴드는 제외
            continue
        crop = img.crop((0, top_px, left_w, bot_px))
        if _ocr_has_keyword(crop):
            # 픽셀 → 포인트 환산 (pt = px / ZOOM)
            top_pt = max(page.rect.y0, (top_px / ZOOM) - ROW_PAD_UP_PT)
            bot_pt = min(page.rect.y1, (bot_px / ZOOM) + ROW_PAD_DOWN_PT)
            return (top_pt, bot_pt)
    return None

def render_page_masking_band(page: fitz.Page, band: Optional[Tuple[float,float]], zoom: float = ZOOM) -> Image.Image:
    mat = fitz.Matrix(zoom, zoom)
    pix = page.get_pixmap(matrix=mat, alpha=False)
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    if band:
        px_per_pt = zoom
        top_px = int(max(0, round(band[0] * px_per_pt)))
        bot_px = int(min(img.height, round(band[1] * px_per_pt)))
        draw = ImageDraw.Draw(img)
        draw.rectangle([(0, top_px), (img.width, bot_px)], fill=(255,255,255))
    return img

In [11]:
pdf_items = list_pdfs_recursive(SOURCE_FOLDER_ID)
print(f"발견된 PDF: {len(pdf_items)}개")

total = 0 # 역시 그냥 지정할 땐 0
t0 = time.time()

for idx, item in enumerate(pdf_items, 1):
    fid, fname = item["id"], item["name"]
    print(f"[{idx}/{len(pdf_items)}] 처리중 → {fname}")

    pdf_bytes = download_pdf_bytes(fid)
    doc = fitz.open(stream=pdf_bytes, filetype="pdf")
    if len(doc) == 0:
        print("  └ 빈 PDF → 스킵")
        doc.close()
        continue

    pages = range(1) if PROCESS_FIRST_PAGE_ONLY else range(len(doc))
    for p in pages:
        page = doc[p]
        band = find_comment_row_band(page)          # Comment 행 위치(없으면 None으로 갈 수 있게 작성, 이거 없으면 두가지 상황 검토 안해주는 듯 ㅠ)
        if band is None:
            print("  └ Comment 행 미검출 → 원본 업로드")
        img = render_page_masking_band(page, band)  # 행 밴드만 흰색으로 지움 덧씌워서 안보이게

        buf = io.BytesIO()
        img.save(buf, format="PNG", optimize=True)
        out_name = f"{fname.rsplit('.pdf',1)[0]}_p{p+1}_"
        upload_png_bytes(buf.getvalue(), out_name, DEST_FOLDER_ID)
        total += 1

    doc.close()

print(f"\n완료! 업로드된 이미지: {total}개")
print(f"저장 폴더(ID): {DEST_FOLDER_ID} | 소요: {time.time()-t0:.1f}s")

발견된 PDF: 13개
[1/13] 처리중 → 25_05_경복궁매시브_고2_토2_2주차_TEST_김준서.pdf
[2/13] 처리중 → 25_05_경복궁매시브_고2_토2_2주차_TEST_김민준.pdf
[3/13] 처리중 → 25_05_경복궁매시브_고2_토2_2주차_TEST_김민기.pdf
[4/13] 처리중 → 25_05_경복궁매시브_고2_토2_2주차_TEST_김승우.pdf
[5/13] 처리중 → 25_05_경복궁매시브_고2_토2_2주차_TEST_주연우.pdf
[6/13] 처리중 → 25_05_경복궁매시브_고2_토2_2주차_TEST_이현준.pdf
[7/13] 처리중 → 25_05_경복궁매시브_고2_토2_2주차_TEST_전우현.pdf
[8/13] 처리중 → 25_05_경복궁매시브_고2_토2_2주차_TEST_배승원.pdf
[9/13] 처리중 → 25_05_경복궁매시브_고2_토2_2주차_HW_이현준.pdf
[10/13] 처리중 → 25_05_경복궁매시브_고2_토2_2주차_HW_주연우.pdf
[11/13] 처리중 → 25_05_경복궁매시브_고2_토2_2주차_HW_배승원.pdf
[12/13] 처리중 → 25_05_경복궁매시브_고2_토2_2주차_HW_김준서.pdf
[13/13] 처리중 → 25_05_경복궁매시브_고2_토2_2주차_HW_김민준.pdf

완료! 업로드된 이미지: 13개
저장 폴더(ID): 1G4ygIz6Dw3ej1M_qj3XNu8Fvtdmn3IRu | 소요: 139.1s
